statistics-resampling in Jupyter Octave Kernel 
============

Interact with Octave and the statistics-resampling package in Notebook. All commands are interpreted by Octave.  Since this is a [MetaKernel](https://github.com/Calysto/metakernel), a standard set of magics are available.  Help on commands is available using the `%help` magic or using `?` with a command.

In [ ]:
% Run this cell first to add the statistics-resampling package (version 5.5.7) functions to path
addpath('package-functions')

In [ ]:
% Enter your code below
